In [ ]:
import pandas as pd
from pandas import DataFrame
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
%matplotlib inline


from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools

from keras.utils.np_utils import to_categorical 
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import *

import os

sns.set(style='white', context='notebook', palette='deep')

In [ ]:
# define train set
from google.colab import drive
drive.mount('/content/drive')
train = pd.read_csv('./drive/My Drive/DACON/data/train.csv')
test = pd.read_csv('./drive/My Drive/DACON/data/test.csv')
train_copy = train.copy()
test_copy = test.copy()

In [ ]:
letter_train = train_copy.iloc[:,3:]
letter_test = test_copy.iloc[:,2:]
x_train = pd.concat([letter_train,letter_test],axis=0).values
x_train = np.divide(x_train,255).reshape(-1,28,28,1)

y = pd.concat([train_copy['letter'],test_copy['letter']],axis=0)
y_train = [ord(x)-65 for x in y]
y_train = to_categorical(y_train,num_classes = 26)

# split training and validation set
x_train, x_val, y_train, y_val = train_test_split(x_train,y_train,test_size=0.1,random_state=25)


In [ ]:
model = Sequential()
base_model = Sequential()
N = 64
base_model.add(Conv2D(filters = N, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (28,28,1)))
base_model.add(Conv2D(filters = N, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'))
base_model.add(MaxPool2D(pool_size=(2,2)))
base_model.add(Dropout(0.25))


base_model.add(Conv2D(filters = 2*N, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
base_model.add(Conv2D(filters = 2*N, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
base_model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
base_model.add(Dropout(0.25))

model.add(base_model)

model.add(Flatten())
model.add(Dense(8*N, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(26, activation = "softmax"))

# define the optimizer
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

# compile the model
model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

MODEL_SAVE_FOLDER_PATH = './drive/My Drive/DACON/model_base/'
if not os.path.exists(MODEL_SAVE_FOLDER_PATH):
  os.mkdir(MODEL_SAVE_FOLDER_PATH)

model_path = MODEL_SAVE_FOLDER_PATH + '{epoch:02d}-{val_accuracy:.4f}.hdf5'

# callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = ModelCheckpoint(filepath = model_path, save_best_only=True, monitor='val_loss', mode='min', verbose=0)
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=0, min_delta=1e-4, mode='min')

epochs = 100
batch_size = 50

In [ ]:
# with data augmentation to prevent overfitting 

datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=2,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.04, # Randomly zoom image 
        width_shift_range=0.08,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.08,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images


In [ ]:
# fit the model
history = model.fit_generator(datagen.flow(x_train,y_train, batch_size=batch_size),
                              epochs = epochs, 
                              validation_data = (x_val,y_val),
                              steps_per_epoch=x_train.shape[0]// batch_size
                              , callbacks=[early_stopping,mcp_save,reduce_lr_loss])

# history = model.fit(x_train, y_train, batch_size=batch_size, epochs = epochs, validation_data = (x_val,y_val))

# plot the loss and accuracy curves for training and validation 
fig, ax = plt.subplots(2,1)
ax[0].plot(history.history['loss'], color='b', label="Training loss")
ax[0].plot(history.history['val_loss'], color='r', label="validation loss",axes =ax[0])
legend = ax[0].legend(loc='best', shadow=True)

ax[1].plot(history.history['accuracy'], color='b', label="Training accuracy")
ax[1].plot(history.history['val_accuracy'], color='r',label="Validation accuracy")
legend = ax[1].legend(loc='best', shadow=True)

In [ ]:
model.summary()

In [ ]:
base_model.summary()

In [ ]:
base_model.trainable = False

In [ ]:
transfer = Sequential()

transfer.add(base_model)

transfer.add(Flatten())
transfer.add(Dense(16*N, activation = "relu"))
transfer.add(Dropout(0.5))
transfer.add(Dense(10, activation = "softmax"))

optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

# compile the model
transfer.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

In [ ]:
transfer.summary()

In [ ]:
rot_gen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    zca_epsilon=1e-06,
    rotation_range=45, # rotation range 1이 최대로 움직인 각도 : 45도
    width_shift_range=0.0,
    height_shift_range=0.0,
    brightness_range=None,
    shear_range=0,     # 블로그 펌 ㅎ
    zoom_range=0,      # 마찬가지 블로그 펌 ㅎ
    channel_shift_range=0.0,
    fill_mode='constant', # 밀린 부분은 0으로 고정
    cval=0.0,             # 밀린 부분에 해당하는 constant
    horizontal_flip=False, # 뒤집기
    vertical_flip=False,   # 뒤집기2
    rescale=1./255, # Rescale
    preprocessing_function=None,
    data_format=None,
    validation_split=0, # Valid split ; 나중에 따로 할 필요없음
    dtype=None
)

trans_gen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    zca_epsilon=1e-06,
    rotation_range=0, # rotation range 1이 최대로 움직인 각도 : 45도
    width_shift_range=0.2, # 블로그 펌
    height_shift_range=0.2,# 블로그 펌
    brightness_range=None,
    shear_range=0,     # 블로그 펌
    zoom_range=0,      # 마찬가지 블로그 펌
    channel_shift_range=0.0,
    fill_mode='constant', # 밀린 부분은 0으로 고정
    cval=0.0,             # 밀린 부분에 해당하는 constant
    horizontal_flip=False, # 뒤집기
    vertical_flip=False,   # 뒤집기2
    rescale=1./255, # Rescale
    preprocessing_function=None,
    data_format=None,
    validation_split=0, # Valid split ; 나중에 따로 할 필요없음
    dtype=None
)

shear_zoom_gen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    zca_epsilon=1e-06,
    rotation_range=0, # rotation range 1이 최대로 움직인 각도 : 45도
    width_shift_range=0.0,
    height_shift_range=0.0,
    brightness_range=None,
    shear_range=0.2,     # 블로그 펌
    zoom_range=0.2,      # 마찬가지 블로그 펌
    channel_shift_range=0.0,
    fill_mode='constant', # 밀린 부분은 0으로 고정
    cval=0.0,             # 밀린 부분에 해당하는 constant
    horizontal_flip=False, # 뒤집기
    vertical_flip=False,   # 뒤집기2
    rescale=1./255, # Rescale
    preprocessing_function=None,
    data_format=None,
    validation_split=0, # Valid split ; 나중에 따로 할 필요없음
    dtype=None
)

flip_gen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    zca_epsilon=1e-06,
    rotation_range=0, # rotation range 1이 최대로 움직인 각도 : 45도
    width_shift_range=0.0,
    height_shift_range=0.0,
    brightness_range=None,
    shear_range=0,     # 블로그 펌
    zoom_range=0,      # 마찬가지 블로그 펌
    channel_shift_range=0.0,
    fill_mode='constant', # 밀린 부분은 0으로 고정
    cval=0.0,             # 밀린 부분에 해당하는 constant
    horizontal_flip=True, # 뒤집기
    vertical_flip=True,   # 뒤집기2
    rescale=1./255, # Rescale
    preprocessing_function=None,
    data_format=None,
    validation_split=0, # Valid split ; 나중에 따로 할 필요없음
    dtype=None
)
df = train_copy
new_data_set = []
num_of_training_set = df.shape[0]

for i in range(num_of_training_set//4):
    rand_1 = np.random.randint(num_of_training_set)
    rand_2 = np.random.randint(num_of_training_set)
    rand_3 = np.random.randint(num_of_training_set)
    rand_4 = np.random.randint(num_of_training_set)
   
    for j in range(5):
        # rotation
        _rot = rot_gen.flow( np.array(df.iloc[rand_1,3:]).reshape(1,28,28,1) ).next().reshape(784,)
        new_data_set += [[
            df.iloc[rand_1,1],
            df.iloc[rand_1,2],
        ] + list(_rot)]
        # translation
        _trans = trans_gen.flow( np.array(df.iloc[rand_2,3:]).reshape(1,28,28,1) ).next().reshape(784,)
        new_data_set += [[
            df.iloc[rand_2,1],
            df.iloc[rand_2,2],
        ] + list(_trans)]
        # shear / zoom
        _shear = shear_zoom_gen.flow( np.array(df.iloc[rand_3,3:]).reshape(1,28,28,1) ).next().reshape(784,)
        new_data_set += [[
            df.iloc[rand_3,1],
            df.iloc[rand_3,2],
        ] + list(_shear)]
        # flip
        _flip = flip_gen.flow( np.array(df.iloc[rand_4,3:]).reshape(1,28,28,1) ).next().reshape(784,)
        new_data_set += [[
            df.iloc[rand_4,1],
            df.iloc[rand_4,2],
        ] + list(_flip)]

In [ ]:
x_train_d = np.divide(train_copy.iloc[:,3:].values,255)
for i in range(len(new_data_set)):
    x_train_d = np.vstack( (x_train_d, np.array(new_data_set[i][2:])) )
x_train_d = x_train_d.reshape(-1,28,28,1)

y_train_d = train_copy['digit'].copy()
for i in range(len(new_data_set)):
    y_train_d = np.append(y_train_d,new_data_set[i][0])
y_train_d = to_categorical(y_train_d,num_classes = 10)

# split training and validation set
x_train_d, x_val_d, y_train_d, y_val_d = train_test_split(x_train_d,y_train_d,test_size=0.1,random_state=15)


In [ ]:
x_train_d = train_copy.iloc[:,3:].copy()
x_train_d = np.divide(x_train_d.values,255).reshape(-1,28,28,1)


y_train_d = train_copy['digit'].copy()
y_train_d = to_categorical(y_train_d,num_classes = 10)

# split training and validation set
x_train_d, x_val_d, y_train_d, y_val_d = train_test_split(x_train_d,y_train_d,test_size=0.1,random_state=45)


In [ ]:
MODEL_SAVE_FOLDER_PATH = './drive/My Drive/DACON/model_transfer/'
if not os.path.exists(MODEL_SAVE_FOLDER_PATH):
  os.mkdir(MODEL_SAVE_FOLDER_PATH)

model_path = MODEL_SAVE_FOLDER_PATH + '{epoch:02d}-{val_accuracy:.4f}.hdf5'

history = transfer.fit(x_train_d,y_train_d, batch_size=batch_size,
                              epochs = epochs, 
                              validation_data = (x_val_d,y_val_d),
                              steps_per_epoch=x_train_d.shape[0]// batch_size
                              , callbacks=[early_stopping,mcp_save,reduce_lr_loss])


# plot the loss and accuracy curves for training and validation 
fig, ax = plt.subplots(2,1)
ax[0].plot(history.history['loss'], color='b', label="Training loss")
ax[0].plot(history.history['val_loss'], color='r', label="validation loss",axes =ax[0])
legend = ax[0].legend(loc='best', shadow=True)

ax[1].plot(history.history['accuracy'], color='b', label="Training accuracy")
ax[1].plot(history.history['val_accuracy'], color='r',label="Validation accuracy")
legend = ax[1].legend(loc='best', shadow=True)